<a href="https://colab.research.google.com/github/voodoohop/pollinations/blob/dev/colabs/pollinator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Code (Frontend and IPFS Connection)

In [ ]:
!killall -9 ipfs

In [ ]:
#@title Install IPFS

!pip install coolname gdown papermill
#!pip install asyncio nest-asyncio coolname aioipfs
#!pip install ipfshttpclient
#!npm install -g nodemon
#!npm install -g voodoohop/pollinations/app


%cd /content
!git clone https://github.com/voodoohop/pollinations.git
%cd /content/pollinations/app
!git pull
!git checkout dev 
!git pull
!npm install
!npm install -g
%cd -
#!npm install forever -g

In [ ]:
#@title Config
from coolname import generate_slug
#ipfs_nodeid = generate_slug(2)#asyncio.run(loop.create_task(get_node_id()))
IPFS_PEER = "ipfs.pollinations.ai"
ipfs_root = "/content/ipfs_root"

print(f"---Settings---\nIPFS_ROOT: {ipfs_root}\nIPFS_PEER: {IPFS_PEER}")

In [ ]:
#@title Empty and create ipfs root: */content/ipfs*



!rm -r $ipfs_root
!mkdir -p $ipfs_root/input
!mkdir -p $ipfs_root/output
!echo "Created IPFS resource" > $ipfs_root/output/log

parameters={"bla":2}
for key, default in parameters.items():
    print(default, file=open(f"{ipfs_root}/input/{key}", 'a'))



In [ ]:
!wget https://dist.ipfs.io/go-ipfs/v0.8.0/go-ipfs_v0.8.0_linux-amd64.tar.gz
!tar -xvzf go-ipfs_v0.8.0_linux-amd64.tar.gz
%cd go-ipfs
!sudo bash install.sh
%cd -
!ipfs init --profile test
#!mkdir -p /ipfs
#!mkdir -p /ipns
!ipfs config Addresses.API /ip4/0.0.0.0/tcp/5001
!ipfs config --json API.HTTPHeaders.Access-Control-Allow-Origin '["*"]'
!nohup ipfs daemon --enable-namesys-pubsub --enable-pubsub-experiment  &
!sleep 15
!ipfs swarm connect /ip4/18.157.205.205/tcp/4001/p2p/12D3KooWDwQ1R9ZmDRv8aWL4dJ4svS9AYvwfJicQ1F45W5aHAVmy



In [ ]:
from os.path import join


notebook_path = join(ipfs_root,"latentvisions.ipynb")
notebook_params_path = join(ipfs_root,"parameters.json")

!gdown --id 1zOA0bUJ68zuZXcbCoN2LPNjnkvxzFpv_ -O $notebook_path

!python pollinations/pollinations/get_params.py $notebook_path > $notebook_params_path

In [ ]:
!echo $ipfs_root
content_id=!ipfs add -r -Q $ipfs_root
content_id=content_id[0]
node_id=!ipfs name publish -Q --offline /ipfs/$content_id
node_id=node_id[0]

#!ipfs pubsub pub $node_id $content_id
print(content_id,node_id)

In [ ]:
#@title Create connection to frontend UI

import IPython
from google.colab import output

display(IPython.display.Javascript(f'''
  const iframe = document.createElement("iframe")
  iframe.src = `http://localhost:3000/incolab.html?node={node_id},{content_id}`

  document.querySelector("#output-area").appendChild(iframe)
'''))



In [37]:
!ipfs pubsub sub $node_id | DEBUG=* pollinate -p /content/ipfs_root --ipns

  ipfsConnector Connecting to IPFS http://ipfs.pollinations.ai:5001 +0ms
  ipfsConnector NodeID Promise { <pending> } +142ms
  ipfsWatch CLI options {
  path: '/content/ipfs_root',
  receive: false,
  send: false,
  once: false,
  ipns: true
} +0ms
  ipfsConnector Creating folder / mfsRoot / +9ms
  ipfsWatch IPFS: Created root IPFS path (if it did not exist) +591ms
  ipfsWatch Local: Watching /content/ipfs_root +0ms
  ipfsWatch Changed files [
  { file: 'output', event: 'addDir' },
  { file: 'input', event: 'addDir' },
  { file: 'output/log', event: 'add' },
  { file: 'input/text_not', event: 'add' },
  { file: 'input/text_input', event: 'add' },
  { file: 'input/num_iterations', event: 'add' },
  { file: 'input/bla', event: 'add' },
  { file: 'parameters.json', event: 'add' },
  { file: 'latentvisions.ipynb', event: 'add' },
  { file: 'README.md', event: 'add' }
] +2s
  ipfsConnector Creating folder output mfsRoot output +2s
  ipfsConnector Creating folder input mfsRoot input +1ms
  i

In [ ]:
async def update_ipfs_content(contentid):
  global ipfs_contentid
  contentid = contentid.decode("utf-8")
  
  await client.get(contentid)
  !cp -rv $contentid/* $ipfs_root
  #!rm -r $contentid
  ipfs_contentid = contentid

content_id=!ipfs name resolve k51qzi5uqu5dh0esiimwcc251a5oq9ivhjppf1zqip1cpdjcvtwj1olbdmqk9k
#on_frontend_data(update_ipfs_content)
print(content_id)

In [ ]:
from glob import glob
from os.path import basename
for path in glob(f"{ipfs_root}/input/*"):
  key = basename(path)
  with open(path,'r') as f:
    value = f.read()
    parameters[key] = value

print("New parameters", parameters)

## DISABLED! CODE (DEEP-DAZE)

In [ ]:
#!pip install --upgrade pip

#!pip install --upgrade ipython
#import papermill as pm
asda
from subprocess import Popen, PIPE, STDOUT
from sys import stdout
import os

#parameters["text"] = "A black and white photo of Saeko with her new Elektron FM synthesizer"
def run(parameters, watch_media_paths="./*.jpg"):
  print("Running",parameters)
 
  for filepath in glob(watch_media_paths):
    print("Deleting",filepath,"before starting.")
    os.remove(filepath)

  _params = [["-p", key, value] for key, value in parameters.items()]
  params = [str(val) for sublist in _params for val in sublist]
  #%run latentvisions.ipynb
  #!papermill "./latentvisions.ipynb" ./output 
  cmd = ["papermill", "/content/latentvisions.ipynb","/content/latentvisions_output.ipynb"] + params

  p = Popen(cmd, stdout = PIPE, 
          stderr = STDOUT, shell = False)

  found_media_files=[]

  while True:
    line = p.stdout.readline()#
    if not line: break

    stdout.write(line)
    with open(f"{ipfs_root}/log","w") as log_out:
      log_out.write(line.decode("utf-8"))
      log_out.flush()
      
    #headers = { 
    #    "params": params,
    #    "text": text
    #}
    #conn.send(body=line.decode("utf-8"), headers=headers, destination='/topic/colabOut')
    #print("new contentid", client.add(ipfs_root, recursive=True))
    
    #continue
    new_found_media_files = glob(watch_media_paths)
    new_found_media_files.sort(key=os.path.getmtime)
    added_media_files = list(set(new_found_media_files)-set(found_media_files))
    found_media_files = new_found_media_files
    pin=False
    if len(added_media_files) > 0:
      for added in added_media_files:
        sleep(0.1)
        if len(added) > 0:
          !cp $added $ipfs_root/output
          print("Adding:", added,"to ipfs root")
          pin=True
    add_and_publish()    
        
while True:
  run(parameters,"./taming_transformers/*.jpg")